In [1]:
import generate_summary.generate_coreference_summary as coref

import pandas as pd

news_list = pd.read_pickle("data/sample_news.pkl")

[nltk_data] Downloading package punkt to /Users/kneskung/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
sample = news_list[5]['content']
sample

"-LRB- CNN -RRB- A Duke student has admitted to hanging a noose made of rope from a tree near a student union , university officials said Thursday . The prestigious private school did n't identify the student , citing federal privacy laws . In a news release , it said the student was no longer on campus and will face student conduct review . The student was identified during an investigation by campus police and the office of student affairs and admitted to placing the noose on the tree early Wednesday , the university said . Officials are still trying to determine if other people were involved . Criminal investigations into the incident are ongoing as well . Students and faculty members marched Wednesday afternoon chanting `` We are not afraid . We stand together , '' after pictures of the noose were passed around on social media . At a forum held on the steps of Duke Chapel , close to where the noose was discovered at 2 a.m. , hundreds of people gathered . `` You came here for the re

In [8]:
result = coref.generate_summary(sample)

**************************************************
sim_mat:  [[0. 0. 0. 3. 1. 1. 0. 0. 0. 0. 2. 1. 2. 2. 3. 2. 1. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 2. 0. 2. 2. 1. 2.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 2. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1.]
 [0. 0. 0. 0. 0. 1. 2. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [2. 0. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 2. 3. 0. 2.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
 [2. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 2. 2. 0. 2.]
 [2. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
 [3. 0. 0. 2. 0. 1. 1. 1. 1. 1. 2. 0. 2. 0. 0. 2. 0. 3.]
 [2. 0. 1. 2. 0. 0. 0. 0. 0. 0. 3. 0. 2. 0. 2. 0. 0. 2.]
 [1. 0. 0. 1. 0. 0. 1. 1. 0

In [9]:
sent = result.split(".")
sent

['-LRB- CNN -RRB- A Duke student has admitted to hanging a noose made of rope from a tree near a student union , university officials said Thursday ',
 " The prestigious private school did n't identify the student , citing federal privacy laws ",
 ' In a news release , it said the student was no longer on campus and will face student conduct review ',
 ' The student was identified during an investigation by campus police and the office of student affairs and admitted to placing the noose on the tree early Wednesday , the university said ',
 ' Students and faculty members marched Wednesday afternoon chanting `` We are not afraid ',
 ' The incident is one of several recent racist events to affect college students ',
 ' A statement issued by Duke said there was a previous report of hate speech directed at students on campus ',
 ' In the news release , the vice president for student affairs called the noose incident a `` cowardly act ',
 " '' Duke University is a private college with about

In [10]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

In [11]:
embeddings = model.encode(sent)

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim) - 1):
    for j in range(i + 1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(sent[i], sent[j], cos_sim[i][j]))

Top-5 most similar pairs:
-LRB- CNN -RRB- A Duke student has admitted to hanging a noose made of rope from a tree near a student union , university officials said Thursday  	  The student was identified during an investigation by campus police and the office of student affairs and admitted to placing the noose on the tree early Wednesday , the university said  	 0.5720
 A statement issued by Duke said there was a previous report of hate speech directed at students on campus  	  '' Duke University is a private college with about 15,000 students in Durham , North Carolina  	 0.4843
 In a news release , it said the student was no longer on campus and will face student conduct review  	  A statement issued by Duke said there was a previous report of hate speech directed at students on campus  	 0.4710
 The incident is one of several recent racist events to affect college students  	  A statement issued by Duke said there was a previous report of hate speech directed at students on campus  